In [ ]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!wget https://www.manythings.org/anki/rus-eng.zip
!unzip rus-eng.zip

--2023-02-19 13:02:35--  https://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15011848 (14M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  14.32M  6.21MB/s    in 2.3s    

2023-02-19 13:02:38 (6.21 MB/s) - ‘rus-eng.zip’ saved [15011848/15011848]

Archive:  rus-eng.zip
  inflating: rus.txt                 
  inflating: _about.txt              


**далее нужно переименовать полученный файл rus.txt в eng-rus.txt для удобства**

In [ ]:
!tail eng-rus.txt

We need to uphold laws against discrimination — in hiring, and in housing, and in education, and in the criminal justice system. That is what our Constitution and our highest ideals require.	Нам нужно отстаивать законы против дискриминации при найме на работу, в жилищной сфере, в сфере образования и правоохранительной системе. Этого требуют наша Конституция и высшие идеалы.	CC-BY 2.0 (France) Attribution: tatoeba.org #5762728 (BHO) & #6390439 (odexed)
I've heard that you should never date anyone who is less than half your age plus seven. Tom is now 30 years old and Mary is 17. How many years will Tom need to wait until he can start dating Mary?	Я слышал, что никогда не следует встречаться с кем-то вдвое младше вас плюс семь лет. Тому 30 лет, a Мэри 17. Сколько лет Тому нужно ждать до тех пор, пока он сможет начать встречаться с Мэри?	CC-BY 2.0 (France) Attribution: tatoeba.org #10068197 (CK) & #10644473 (notenoughsun)
I do have one final ask of you as your president, the same thing I a

In [ ]:
sep = 'CC-BY'
with open('/content/eng-rus.txt') as file:
    for line in file:
        print(line.split(sep, 1)[0])

Выходные данные были обрезаны до нескольких последних строк (5000).
Tom might come.	Том может прийти.	
Tom might talk.	Том может заговорить.	
Tom might talk.	Том может всё рассказать.	
Tom might wait.	Том может подождать.	
Tom missed you.	Том по вам скучал.	
Tom missed you.	Том скучал по тебе.	
Tom missed you.	Том по тебе скучал.	
Tom misses you.	Том скучает по тебе.	
Tom misses you.	Том по Вам скучает.	
Tom misses you.	Том по вам скучает.	
Tom misses you.	Том по тебе скучает.	
Tom must do it.	Том должен это сделать.	
Tom must leave.	Том должен уйти.	
Tom must leave.	Том должен уехать.	
Tom needed you.	Ты был нужен Тому.	
Tom needed you.	Ты была нужна Тому.	
Tom needed you.	Вы были нужны Тому.	
Tom needed you.	Том нуждался в тебе.	
Tom needed you.	Том в вас нуждался.	
Tom needs Mary.	Тому нужна Мэри.	
Tom needs Mary.	Мэри нужна Тому.	
Tom needs food.	Тому нужна еда.	
Tom needs help.	Тому нужна помощь.	
Tom needs help.	Том нуждается в помощи.	
Tom needs hope.	Тому нужна надежда.	
Tom ne

KeyboardInterrupt: ignored

# **1 словарь языка**

In [ ]:
SOS_token = 0
EOS_token = 1

# объект, который позволит работать с языком
class Lang:
    def __init__(self, name):
        self.name = name # сохраняется имя
        self.word2index = {} # перевод слов в индексы
        self.word2count = {} # счетчик одинаковых слов
        self.index2word = {0: "SOS", 1: "EOS"} # добавляет 2 токена, делает обратное преобразование из индексов в цифры
        self.n_words = 2  # Count SOS and EOS (количество слов, которое мы используем в нашем словаре)

    def addSentence(self, sentence): # функция: добавить предложение
        for word in sentence.split(' '): # каждое предложение разбиваем по пробелам, получаем слова
            self.addWord(word) # каждое слово добавляем в словарь

    def addWord(self, word): # функция: добавления слова
        if word not in self.word2index: # проверяем находится ли слово внутри словаря
            self.word2index[word] = self.n_words # добавляем новое слово, а именно, в форме: ключ-значение (пример: {239:'зонтик'})
            self.word2count[word] = 1
            self.index2word[self.n_words] = word # 'зонтик': 239
            self.n_words += 1
        else:
            self.word2count[word] += 1 # если слово в словаре, то добавляем количество данных слов в словаре

# **2 функция с кодировкой unicodeToAscii(s):**
# **3 функция, которая приводит в порядок текст normalizeString(s):**

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = s.lower()
    s = re.sub('[.!?]','',s)
#    s = re.sub(r"([.!?])", r" \1", s)
#    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

# **4 функция, которая читает наш словарь:**

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # читаем файл и разбиваем на линии (строки по \n)
    lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')
    lines = [i.split('\tCC-BY', 1)[0] for i in lines]
    # разбивает все линии на пары и очищает, т.е. делает: def normalizeString(s)(см. выше)
    # т.к. у нас идет: предложение, затем предложение-перевод, то здесь ведется обработка предложений - парами.
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Реверс, который меняет языки местами (т.е. меняетпредложения попарно местами)
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

# **5 Сокращаем текст (способ: указываем слова с которых должны начинаться предложения eng_prefixes)**

In [ ]:
MAX_LENGTH = 20

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# **6 Функция считает наши словари**

In [ ]:
# читает словари, фильтрует пары, печатает сколько осталось,
# добавляет пары в соответствующие языки, выводим статистику
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'rus', True)
print(random.choice(pairs))

Reading lines...
Read 451436 sentence pairs
Trimmed to 4819 sentence pairs
Counting words...
Counted words:
rus 4657
eng 2401
['я очень ленивая', 'i am really lazy']


# **7 The Encoder** (прогоняет наши данные через модель)
-----------





In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self): # обнуляет скрытое состояние
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class EncoderRNN2(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers, rnn_type):
        super(EncoderRNN2, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = rnn_type(hidden_size, hidden_size, n_layers)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.rnn(output, hidden)
        return output, hidden

    def initHidden(self, n_layers):
        return torch.zeros(n_layers, 1, self.hidden_size, device=device)

#  **8 The Decoder**
-----------




In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN2(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers, rnn_type):
        super(DecoderRNN2, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.rnn       = rnn_type(hidden_size, hidden_size, n_layers)
        self.out       = nn.Linear(hidden_size, output_size)
        self.softmax   = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output         = self.embedding(input).view(1, 1, -1)
        output         = F.relu(output)
        output, hidden = self.rnn(output, hidden)
        output         = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self, n_layers):
        return torch.zeros(n_layers, 1, self.hidden_size, device=device)

# **9 Функции которые переводят наши пары в тензоры:**

# !!!!!!!!!!в этом уроке каждый индекс - это слово!!!!!!!!!

In [ ]:
# получаем индексы из предложений
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

# функция использует функцию выше (предыдущую)
def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

# функция использует функцию выше (предыдущую)
def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

# **10 функция train обучает модель на одном предложении:**

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

# **11 функции работы со временем**

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

# **12 функция, которая обучает нашу модель на всех предложениях**

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    # список тренировочных пар:
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

# **13 отрисовывается график lossов**

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

# **14 функция, которая будет генерировать предложение**

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    # max_length=MAX_LENGTH - это задается максимальная длина сгенерированного предложения
    with torch.no_grad(): # отключаем градиенты, т.к. они уже обучены
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words # на выходе словарь из слов последовательности

# **15 Функция выводит заданное (n=10) число предложений и их переводов**

# **nn.GRU - один слой**

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

0m 49s (- 11m 28s) (5000 6%) 3.2013
1m 34s (- 10m 16s) (10000 13%) 2.7371
2m 21s (- 9m 26s) (15000 20%) 2.3026
3m 8s (- 8m 39s) (20000 26%) 1.8886
3m 56s (- 7m 52s) (25000 33%) 1.5629
4m 44s (- 7m 6s) (30000 40%) 1.2314
5m 32s (- 6m 20s) (35000 46%) 1.0067
6m 20s (- 5m 33s) (40000 53%) 0.7897
7m 9s (- 4m 46s) (45000 60%) 0.6158
7m 57s (- 3m 58s) (50000 66%) 0.4620
8m 45s (- 3m 10s) (55000 73%) 0.3530
9m 33s (- 2m 23s) (60000 80%) 0.2639
10m 22s (- 1m 35s) (65000 86%) 0.1989
11m 11s (- 0m 47s) (70000 93%) 0.1473
12m 0s (- 0m 0s) (75000 100%) 0.1136


In [ ]:
evaluateRandomly(encoder1, decoder1)

> они завидуют нашему успеху
= they are jealous of our success
< they are jealous of our success <EOS>

> с ним очень трудно поладить
= he is very difficult to get along with
< he is very difficult to get along with <EOS>

> он умнее меня
= he is more clever than i am
< he is more clever than me <EOS>

> мы измеряем глубину реки
= we are measuring the depth of the river
< we are measuring the depth of the river <EOS>

> я обедаю со своею сестрои
= i am eating lunch with my sister
< i am eating lunch with my sister <EOS>

> она - второкурсница
= she is a second year student
< she is a second <EOS>

> он знаменитыи композитор
= he is a famous composer
< he is a famous composer <EOS>

> я состою в теннисном клубе
= i am a member of the tennis club
< i am a member of the tennis club <EOS>

> она осталась дома одна
= she stayed at home by herself
< she stayed at home by herself <EOS>

> он твердыи орешек
= he is a tough cookie
< he is a tough cookie <EOS>



# **nn.GRU - два слоя**

# **7 The Encoder** (прогоняет наши данные через модель)
-----------





In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)#, num_layers=2)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self): # обнуляет скрытое состояние
        return torch.zeros(1, 1, self.hidden_size, device=device)

#  **8 The Decoder**
-----------




In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)#, num_layers=2)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)
trainIters(encoder1, decoder1, 75000, print_every=5000)

0m 48s (- 11m 20s) (5000 6%) 3.2219
1m 32s (- 10m 4s) (10000 13%) 2.7357
2m 18s (- 9m 13s) (15000 20%) 2.3259
3m 4s (- 8m 26s) (20000 26%) 1.9263
3m 50s (- 7m 40s) (25000 33%) 1.5500
4m 38s (- 6m 57s) (30000 40%) 1.2803
5m 24s (- 6m 10s) (35000 46%) 1.0044
6m 11s (- 5m 24s) (40000 53%) 0.8100
6m 57s (- 4m 38s) (45000 60%) 0.6499
7m 44s (- 3m 52s) (50000 66%) 0.5064
8m 31s (- 3m 6s) (55000 73%) 0.3862
9m 18s (- 2m 19s) (60000 80%) 0.2881
10m 5s (- 1m 33s) (65000 86%) 0.2000
10m 51s (- 0m 46s) (70000 93%) 0.1519
11m 38s (- 0m 0s) (75000 100%) 0.1130


In [ ]:
evaluateRandomly(encoder1, decoder1)

> я жду письма от нее
= i am expecting a letter from her
< i am expecting a letter from her <EOS>

> вы должны сдать домашнее задание до конца недели
= you are supposed to hand in the homework by the end of the week
< you are supposed to hand in the homework by the end of the week <EOS>

> она должна родить в этом месяце
= she is going to have a baby this month
< she is going to have a baby this month <EOS>

> ты слишком молода, чтобы путешествовать однои
= you are too young to travel alone
< you are too young to travel alone <EOS>

> он вылитыи отец
= he is the spitting image of his father
< he is the spitting image of his father <EOS>

> я не интересуюсь литературои так, как ты
= i am not as interested in literature as you
< i am not as interested in literature as you <EOS>

> она не такои человек, как ты думаешь
= she is not the kind of person you think she is
< she is not the kind of person you think she is <EOS>

> он, наверное, придет
= he is likely to come
< he is likely to come

# **nn.LSTM - один слой**

# **7 The Encoder** (прогоняет наши данные через модель)
-----------





In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        #self.gru = nn.GRU(hidden_size, hidden_size, num_layers = 2) # здесь добавляем доп слой GRU
        self.lstm = nn.LSTM(input_size, hidden_size)#, batch_first=True)#, num_layers = 2) # здесь добавляем доп слой LSTM
#input_size=input_size, hidden_size=hidden_size, num_layers=num_layers,
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.lstm(output, hidden) #lstm with input, hidden, and internal state
#        output, hidden = self.LSTM(output, hidden)
        return output, hidden[0]

    def initHidden(self): # обнуляет скрытое состояние
        return torch.zeros(1, 1, self.hidden_size, device=device)

#  **8 The Decoder**
-----------




In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        #self.gru = nn.GRU(hidden_size, hidden_size, num_layers = 2) # здесь добавляем доп слой GRU
        self.lstm = nn.LSTM(hidden_size, hidden_size)#, batch_first=True)#, num_layers = 2) # здесь добавляем доп слой LSTM
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
#        hid1 = hidden[0]
#        hid2 = hidden[1]
#        output, (hidden1, hidden2) = self.lstm(output, (hid1, hid2))
        output, hidden = self.lstm(output, hidden)
        output = self.softmax(self.out(output))
        return output, hidden[0]

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

IndexError: ignored

In [ ]:
evaluateRandomly(encoder1, decoder1)

> они завидуют нашему успеху
= they are jealous of our success
< they are jealous of our success <EOS>

> с ним очень трудно поладить
= he is very difficult to get along with
< he is very difficult to get along with <EOS>

> он умнее меня
= he is more clever than i am
< he is more clever than me <EOS>

> мы измеряем глубину реки
= we are measuring the depth of the river
< we are measuring the depth of the river <EOS>

> я обедаю со своею сестрои
= i am eating lunch with my sister
< i am eating lunch with my sister <EOS>

> она - второкурсница
= she is a second year student
< she is a second <EOS>

> он знаменитыи композитор
= he is a famous composer
< he is a famous composer <EOS>

> я состою в теннисном клубе
= i am a member of the tennis club
< i am a member of the tennis club <EOS>

> она осталась дома одна
= she stayed at home by herself
< she stayed at home by herself <EOS>

> он твердыи орешек
= he is a tough cookie
< he is a tough cookie <EOS>



# **nn.LSTM - два слоя**

# **7 The Encoder** (прогоняет наши данные через модель)
-----------





In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        #self.gru = nn.GRU(hidden_size, hidden_size, num_layers = 2) # здесь добавляем доп слой GRU
        self.LSTM = nn.LSTM(hidden_size, hidden_size, hidden_size)#, num_layers = 2) # здесь добавляем доп слой LSTM

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.LSTM(output, hidden)
        return output, hidden

    def initHidden(self): # обнуляет скрытое состояние
        return torch.zeros(1, 1, self.hidden_size, device=device)

#  **8 The Decoder**
-----------




In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        #self.gru = nn.GRU(hidden_size, hidden_size, num_layers = 2) # здесь добавляем доп слой GRU
        self.LSTM = nn.LSTM(hidden_size, hidden_size, hidden_size)#, num_layers = 2) # здесь добавляем доп слой LSTM
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.LSTM(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

0m 51s (- 11m 56s) (5000 6%) 3.2595
1m 37s (- 10m 34s) (10000 13%) 2.7471
2m 24s (- 9m 39s) (15000 20%) 2.3102
3m 13s (- 8m 50s) (20000 26%) 1.9221
4m 1s (- 8m 2s) (25000 33%) 1.5792
4m 49s (- 7m 14s) (30000 40%) 1.2668
5m 38s (- 6m 26s) (35000 46%) 1.0031
6m 26s (- 5m 38s) (40000 53%) 0.7752
7m 15s (- 4m 50s) (45000 60%) 0.5944
8m 4s (- 4m 2s) (50000 66%) 0.4626
8m 53s (- 3m 14s) (55000 73%) 0.3573
9m 43s (- 2m 25s) (60000 80%) 0.2616
10m 33s (- 1m 37s) (65000 86%) 0.2060
11m 22s (- 0m 48s) (70000 93%) 0.1414
12m 11s (- 0m 0s) (75000 100%) 0.1116


In [ ]:
evaluateRandomly(encoder1, decoder1)

> они завидуют нашему успеху
= they are jealous of our success
< they are jealous of our success <EOS>

> с ним очень трудно поладить
= he is very difficult to get along with
< he is very difficult to get along with <EOS>

> он умнее меня
= he is more clever than i am
< he is more clever than me <EOS>

> мы измеряем глубину реки
= we are measuring the depth of the river
< we are measuring the depth of the river <EOS>

> я обедаю со своею сестрои
= i am eating lunch with my sister
< i am eating lunch with my sister <EOS>

> она - второкурсница
= she is a second year student
< she is a second <EOS>

> он знаменитыи композитор
= he is a famous composer
< he is a famous composer <EOS>

> я состою в теннисном клубе
= i am a member of the tennis club
< i am a member of the tennis club <EOS>

> она осталась дома одна
= she stayed at home by herself
< she stayed at home by herself <EOS>

> он твердыи орешек
= he is a tough cookie
< he is a tough cookie <EOS>



#**ИТОГО:**

**значение loss:**

In [ ]:
import pandas as pd

In [ ]:
data = {'25':[0.109, 0.247, 0.389], '70':[0.256, 0.241, 0.339], '120':[0.266, 0.245, 0.345]}
df = pd.DataFrame(data, index =['RNN', 'LSTM', 'GRU'])

In [ ]:
df

,25,70,120
RNN,0.109,0.256,0.266
LSTM,0.247,0.241,0.245
GRU,0.389,0.339,0.345
